In [5]:
import sys
import json
import os
# To import python scripts from other folders
sys.path.append('../')
sys.path.append('../../../py/')
from GitManager import GitManager, DELIMITER
from ProjectTestAnalysis import ProjectTestAnalysis
import pandas as pd
root="/home/jovyan/work"
procesed_results_path=root+"/notebooks/ProjectAnalysis/TestAnalysis/results/"

In [6]:
def getProjects(dataset):
    path = root+"/configFiles/%sProjects/"%dataset
    projects = []
    for configFile in os.listdir(path):
        with open(path+configFile) as f:
            project_info = json.load(f)
            if os.path.isdir(procesed_results_path+project_info["project"]):
                projects.append((dataset, project_info["project"]))
    return projects
many4j = getProjects("ManySStub4J")
all_datasets = many4j

In [13]:
def fix_commits(project):
    pa = ProjectTestAnalysis(project,  2, root=root)
    
    report_commits = pa.report.to_dict('records')
    new_report_commits = []
    
    summary_commits = pa.summary.to_dict('records')
    new_summary_commits = []
    
    base_commit = report_commits[-1]['commit']
    
    raw_commit_log_list = GitManager(root+"/projects/"+project, base_commit).getAllCommits()
    commit_list = [ c.split(DELIMITER)[0].replace('"', '') for c in raw_commit_log_list]
    
    for idx, commit_hash in enumerate(commit_list):
        
        commit_info_report = report_commits[idx]
        commit_info_summary = summary_commits[idx]
        
        if not commit_hash.startswith(commit_info_report['commit']): 
            print(commit_hash)
            raise Exception("Error: " + project + " - " + commit_info_report['commit'] + " - " + str(idx))
        
        commit_info_report['commit'] = commit_hash
        new_report_commits.append(commit_info_report)
        
        commit_info_summary['commit'] = commit_hash
        new_summary_commits.append(commit_info_summary)
        
    pd.DataFrame(new_report_commits).to_csv(procesed_results_path+project+"/report.csv", index=False)
    pd.DataFrame(new_summary_commits).to_csv(procesed_results_path+project+"/summary.csv", index=False)
    
fix_commits("spring-boot")

In [ ]:
for _, project in all_datasets:
    try:
          fix_commits(project)
    except Exception as e:
          print(e)

In [10]:
def fix_commits2(project):
    pa = ProjectTestAnalysis(project,  2, root=root)
    
    report_commits = pa.report.to_dict('records')
    new_report_commits = []
    
    summary_commits = pa.summary.to_dict('records')
    new_summary_commits = []
    
    base_commit = report_commits[-1]['commit']
    
    raw_commit_log_list = GitManager(root+"/projects/"+project, base_commit).getAllCommits()
    commit_list = [ c.split(DELIMITER)[0].replace('"', '') for c in raw_commit_log_list]
    
    commit_list_map = {}
    for _hash in commit_list:
        hash8 = _hash[0:8]
        if hash8 not in commit_list_map:
            commit_list_map[hash8] = []
        else:
            print("Conflict")
        commit_list_map[hash8].append(_hash)
        
        if len(commit_list_map[hash8]) > 1:
            print(commit_list_map[hash8])
        
    
    for idx, commit_info_report in enumerate(report_commits):
        
        commit_info_summary = summary_commits[idx]
        
        hash8 = commit_info_report['commit']
        
        commit_hash = hash8
        
        if hash8 in commit_list_map:
            commit_hash = commit_list_map[hash8][0]
        else: 
            pass#print("Commit not longer in git: %s"%hash8)
        
        
        commit_info_report['commit'] = commit_hash
        new_report_commits.append(commit_info_report)
        
        commit_info_summary['commit'] = commit_hash
        new_summary_commits.append(commit_info_summary)
        
#     pd.DataFrame(new_report_commits).to_csv(procesed_results_path+project+"/report_new.csv", index=False)
#     pd.DataFrame(new_summary_commits).to_csv(procesed_results_path+project+"/summary_new.csv", index=False)

In [11]:
fix_commits2("neo4j")

Conflict
['df268902ff11c1d318302bd3affc1eae92eefbe1', 'df268902a073c725dc59dac3da42d93c87de7e42']


In [ ]:
fix_commits2("dropwizard")

In [ ]:
fix_commits2("jedis")

In [ ]:
fix_commits2("antlr4")